# Cours TAL – Labo 4
Nathan Gonzalez Montes et Vincent Guidoux

## Exercice 1 - exécuter la NER dans NLTK

### En utilisant nltk.ne_chunk (voir livre NLTK p. 283 - 4),extraire les entités nommées les plus fréquentes  d’un texte de votre choix

In [1]:
import sys
!{sys.executable} -m pip install numpy

In [2]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
from nltk import FreqDist
FreqDist
from urllib import request
ne_chunk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nortalle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nortalle\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Nortalle\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
url1 = "http://www.gutenberg.org/files/18488/18488.txt"

response = request.urlopen(url1)
raw = response.read().decode('utf8')
raw = raw[4340:489067]

In [4]:
sentences = nltk.sent_tokenize(raw)

sentences = [nltk.word_tokenize(sent) for sent in sentences]

len(sentences)

6701

In [5]:
sentences = [nltk.pos_tag(sent) for sent in sentences]
    
len(sentences)

6701

In [6]:
named_entities = []

for sentence in sentences:
    for chunk in ne_chunk(sentence):
        if hasattr(chunk, 'label'):
            named_entities.append((chunk.label(), ' '.join(c[0] for c in chunk)))

In [7]:
freq_named_entities_nltk = FreqDist(named_entities) 

freq_named_entities_nltk.most_common()

[(('PERSON', 'Priscilla'), 227),
 (('PERSON', 'Farwell'), 194),
 (('GPE', 'Priscilla'), 193),
 (('PERSON', 'Ledyard'), 106),
 (('PERSON', 'Master Farwell'), 64),
 (('ORGANIZATION', 'Boswell'), 61),
 (('ORGANIZATION', 'Travers'), 59),
 (('PERSON', 'Nathaniel'), 55),
 (('GPE', 'Kenmore'), 55),
 (('PERSON', 'Margaret'), 52),
 (('GPE', 'Boswell'), 47),
 (('PERSON', 'Dick'), 42),
 (('ORGANIZATION', 'Priscilla'), 41),
 (('ORGANIZATION', 'McAlpin'), 40),
 (('ORGANIZATION', 'Farwell'), 39),
 (('PERSON', 'Glenn'), 38),
 (('ORGANIZATION', 'States'), 33),
 (('PERSON', 'Priscilla Glenn'), 31),
 (('PERSON', 'Doctor Ledyard'), 31),
 (('PERSON', 'Mary'), 31),
 (('PERSON', 'Margaret Moffatt'), 31),
 (('PERSON', 'Helen'), 30),
 (('PERSON', 'Miss Glynn'), 27),
 (('PERSON', 'Kenmore'), 26),
 (('PERSON', 'Mary McAdam'), 26),
 (('PERSON', 'Jerry'), 25),
 (('PERSON', 'Travers'), 23),
 (('PERSON', 'Huntter'), 23),
 (('PERSON', 'God'), 22),
 (('PERSON', 'Tom'), 21),
 (('PERSON', 'Sandy'), 20),
 (('GPE', 'Farw

### En vous inspirant du POS de Stanford, utiliser leur NER (https://nlp.stanford.edu/software/CRF-NER.html) et la classe StanfordNERTagger de NLTK, pour extraire les entités nommées du même texte

### Comparez la liste des NE les plus fréquentes et leurs types reconnus

## Exercice 2


[FreqDist](https://stackoverflow.com/questions/4634787/freqdist-with-nltk)